In [1]:
!pip install -q -U transformers bitsandbytes peft accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.7 MB/s eta 0:00:00


In [2]:
import re
import os
import torch
from bitsandbytes.nn import modules

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    logging
)

from peft import (
    PeftModel,
    PeftConfig
)

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
auth_token = os.getenv("HF_TOKEN")

In [5]:
base_model = "mistralai/Mistral-7B-Instruct-v0.2"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    token=auth_token,
    padding_side="right",
    add_eos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
text_gen_model = "Mesutby/mistral-7B-wikitext-finetuned"
config = PeftConfig.from_pretrained(text_gen_model)

adapter_config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    use_auth_token=auth_token,
    return_dict=True,
    load_in_4bit=True,
    device_map='auto'
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [9]:
fine_tuned_model = PeftModel.from_pretrained(model, text_gen_model)

pipe = pipeline(task="text-generation",
                model=fine_tuned_model,
                tokenizer=tokenizer,
                eos_token_id=model.config.eos_token_id,
                max_new_tokens=100)

adapter_model.safetensors:   0%|          | 0.00/609M [00:00<?, ?B/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM'

In [12]:
modules.bnb_4bit_compute_dtype = torch.float16

In [13]:
print("Press exit, quit, or q to finish the speech.")
while True:
    prompt = input("Me: ")
    if prompt.lower() in ['exit', 'quit', 'q']:
        print("Thanks for the conversation")
        break

    result = pipe(prompt)
    generated = result[0]['generated_text']

    if generated.lower().startswith(prompt.lower()):
        generated = generated[len(prompt):].strip()

    cleaned_generated = re.sub(r'\s+', ' ', generated).strip()

    print("User Prompt: ", prompt)
    print("Bot:", cleaned_generated)


Press exit, quit, or q to finish the speech.
Me: Do you think artificial intelligence will try to take over the world in the future?
User Prompt:  Do you think artificial intelligence will try to take over the world in the future?
Bot: I don't think it will. I think it will be a tool for us to use to make our lives easier and more efficient. It will be a tool for us to use to solve problems that are too complex for us to solve on our own. It will be a tool for us to use to create new things and to explore new frontiers. But it will not be a threat to us. It will not try to take over the world. It will not try to
Me:  Who is Albert Einstein?
User Prompt:   Who is Albert Einstein?
Bot: The name of the man who is most famous for his theory of relativity.
Me: What is artificial intelligence?
User Prompt:  What is artificial intelligence?
Bot: The term is often used to describe a computer system or a computer program that is designed to perform tasks that would normally require human intell